# Repaso 6 – Pruebas de Hipótesis
Ejemplos ampliados con interpretación profunda.

In [ ]:
import numpy as np
import scipy.stats as stats

## Prueba Z de una media (σ conocida)

In [ ]:
np.random.seed(123)
sueldos = np.random.normal(52000,1800,120)
media=np.mean(sueldos)
z=(media-50000)/(1800/np.sqrt(120))
p=1-stats.norm.cdf(z)
media,z,p

## Prueba t de una media

In [ ]:
from scipy.stats import ttest_1samp
t_stat,p_val=ttest_1samp(sueldos,50000)
t_stat,p_val

## Prueba Chi-cuadrado

In [ ]:
import pandas as pd
tabla=pd.DataFrame({"Compra Sí":[25,15],"Compra No":[10,30]},index=["Producto A","Producto B"])
chi2,p_chi,_,_=stats.chi2_contingency(tabla)
chi2,p_chi

## ANOVA

In [ ]:
area1=np.random.normal(70,8,30)
area2=np.random.normal(75,7,30)
area3=np.random.normal(78,6,30)
f,p=stats.f_oneway(area1,area2,area3)
f,p

# Conclusiones
Este cuaderno contiene ejemplos completos para repaso.